# QTreeView


Unlike `QTreeWidget`, **QTreeView** takes data from a model and build the tree view for you. In a QTreeWidget, you need to build the root node and child all by yourself.

`QTreeView` implements the model-view paradigms. It extends the `QAbstractItemView` and implements the abstract method, so it can render all data model which extends `QAbstractItemModel`. For each `item` in the model, it will render the corresponding cell in the view.

Below are some useful method of the QTreeView class:
- setModel() : associate a model to the view
- setHeader(): setup column name(table header) of the view
- header(): get the object of the header
- indexAbove(): get the index above the current selected row index
- indexBelow(): get the index below the current selected row index
- collapse(): collapse the selected row to hide the child nodes
- collapseAll(): collapse all rows
- expand(): expand the selected row to show the child nodes
- expandAll(): expand all rows

## 1. A simple example

In below example, we build a tree view and a **QStandardItemModel**. Then we add the model to the tree view.

> The QTreeView can take many model, below figure shows all supported model

![qt_model_hierarchy.png](../../../image/qt_model_hierarchy.png)

Each model has pro and cons, for example, the `QStandardItemModel`:

pros:
1. Easy to use, no need to create custom class to store data
2. Can use `appendRow, appendColumn` to build/visualize `list, table, tree` data structures

cons:
1. Performance issues when data become important
2. Consume too many memory

The row column index of the tree view/model

Below figure shows an example of the tree model index
![qt_model_index.png](../../../image/qt_model_index.png)


In [ ]:
import sys

from PyQt6.QtGui import QStandardItemModel, QStandardItem
from PyQt6.QtWidgets import QMainWindow, QTreeView, QApplication


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.treeView = None
        self.initUI()

    def initUI(self):
        # create a tree view
        self.treeView = QTreeView()

        # create a model
        model = QStandardItemModel()
        # populate the model with rows
        for row in range(4):
            item = QStandardItem(str(row))
            model.appendRow(item)


        self.treeView.setModel(model)
        self.treeView.show()
        self.treeView.setWindowTitle("SimpleTree")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainView=MainWindow()
    sys.exit(app.exec())



## 2 Customize a model

In below example, we define our own model (`MyTreeModel`) and item (`MyItem`) of the model.

In [ ]:
import sys

import typing
from PyQt6.QtCore import QAbstractListModel, QModelIndex, Qt
from PyQt6.QtWidgets import QTreeView, QApplication, QStyledItemDelegate, QWidget, QVBoxLayout


class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.createTree()

    def createTree(self):
        data = ['A', 'B', "C", 'D', "E", "F"]
        model = MyTreeModel(data)

        # create a tree view
        self.treeView = QTreeView()

        self.treeView.setItemDelegate(BoldDelegate())
        self.treeView.setModel(model)
        self.treeView.setWindowTitle("SimpleTree")
        layout= QVBoxLayout()
        layout.addWidget(self.treeView)

        self.setLayout(layout)


class MyItem:
    def __init__(self, name, checked):
        self.name = name
        self.checked = checked


class BoldDelegate(QStyledItemDelegate):
    def paint(self, painter, option, index):
        option.font.setWeight(22)
        QStyledItemDelegate.paint(self, painter, option, index)


class MyTreeModel(QAbstractListModel):
    def __init__(self, args, parent=None):
        super(MyTreeModel, self).__init__(parent)

        self.args = []
        for item_name in args:
            self.args.append(MyItem(item_name, False))

    def rowCount(self, parent: QModelIndex = ...) -> int:
        return len(self.args)

    def headerData(self, section: int, orientation: Qt.Orientation, role: int = ...) -> typing.Any:
        """
        set header of the table in the tree view
        Parameters
        ----------
        section
        orientation
        role

        Returns
        -------

        """
        if role == Qt.ItemDataRole.DisplayRole:
            if orientation == Qt.Orientation.Horizontal:
                return "Nodes"

    def flags(self, index: QModelIndex) -> Qt.ItemFlag:
        return Qt.ItemFlag.ItemIsUserCheckable | Qt.ItemFlag.ItemIsEditable | Qt.ItemFlag.ItemIsSelectable \
               | Qt.ItemFlag.ItemIsEnabled

    def data(self, index: QModelIndex, role: int = ...) -> typing.Any:
        if role == Qt.ItemDataRole.DisplayRole:
            row = index.row()
            return self.args[row].name

        if role == Qt.ItemDataRole.CheckStateRole:
            row = index.row()
            if self.args[row].checked:
                return Qt.CheckState.Checked
            else:
                return Qt.CheckState.Unchecked

    def setData(self, index: QModelIndex, value: typing.Any, role: int = ...) -> bool:
        if role == Qt.ItemDataRole.CheckStateRole:
            row = index.row()
            self.args[row].checked = not self.args[row].checked
        return True


if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainView = MainWindow()
    mainView.show()
    sys.exit(app.exec())
